## Part 1 - Data Preparation and Exploration

In [35]:
%%capture
# Due to the configuration of the base Jupter image, the following imports are required for the regressions in the assignment to report the correct metrics

import sys
# !{sys.executable} -m pip uninstall statsmodels --yes
# !{sys.executable} -m pip uninstall numpy --yes
# !{sys.executable} -m pip uninstall pandas --yes
# !{sys.executable} -m pip uninstall patsy --yes
# !{sys.executable} -m pip install numpy==1.17
# !{sys.executable} -m pip install pandas==1.0
# !{sys.executable} -m pip install patsy==0.5.2
# !{sys.executable} -m pip install statsmodels==0.11.1

In [36]:
# Import Libraries

import pandas as pd
import datetime as dt
import scipy.stats as sp
import numpy as np
import statsmodels.formula.api as sm

In [37]:
# 1.	Import the “Shotlog_14_15.csv” data file as “Shotlog_1415” into Jupyter Notebook.
# Import “Player_Stats_1415.csv” data file as “Player_Stats” into Jupyter Notebook. Descriptions of the datasets and selected variables
# a) In the dataset “Shotlog_14_15,” each observation represents an attempt of a shot. In the dataset “Player_Stats_14_15,” each observation represents a player.
# b) The “average_hit” variable in both dataframes indicate the average success rate of a player making a shot over the season. It is defined and calculated the same way in both dataframes.
# c) The variable “home_away” indicates whether the team that the player belongs to played at home or away.
# d) The variable “result” indicates whether the team that the player belongs to won or lost the game. The variable “final_margin” represents the difference in final score between the team the player belongs to and their opponent’s.
# e) The variable “shot_number” is the order of the shot the given player attempted at the given game.
# f) “game_clock” is the countdown clock for each quarter. The game clock starts at 12 minutes. “shot_clock” refers to the display of a countdown clock of the time within which the team possessing the ball must attempt a field goal. The shot clock starts at 24 seconds.

In [38]:
# Import Shotlog_14_15 and Player_Stats Datasets

# Shotlog_1415=pd.read_csv("Assignment Data/Week 6/Shotlog_14_15.csv")
# Player_Stats=pd.read_csv("Assignment Data/Week 6/Player_Stats_14_15.csv")

Shotlog_1415=pd.read_csv("/content/Shotlog_14_15.csv")
Player_Stats=pd.read_csv("/content/Player_Stats_14_15.csv")

display(Shotlog_1415)

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,closest_defender,closest_defender_id,closest_def_dist,current_shot_hit,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game
0,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,1,1,...,"Lopez, Brook",201572,6.6,1,2,al horford,201143,0.541259,715,10
1,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,2,1,...,"Lopez, Brook",201572,5.6,0,0,al horford,201143,0.541259,715,10
2,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,3,1,...,"Lopez, Brook",201572,4.7,0,0,al horford,201143,0.541259,715,10
3,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,4,1,...,"Lopez, Brook",201572,5.8,0,0,al horford,201143,0.541259,715,10
4,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,5,2,...,"Lopez, Brook",201572,6.4,0,0,al horford,201143,0.541259,715,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128064,21400350,14-Dec-14,WAS vs. UTA,WAS,UTA,H,W,9,11,3,...,"Burke, Trey",203504,4.7,1,2,john wall,202322,0.448513,874,15
128065,21400350,14-Dec-14,WAS vs. UTA,WAS,UTA,H,W,9,12,3,...,"Exum, Dante",203957,3.4,1,2,john wall,202322,0.448513,874,15
128066,21400350,14-Dec-14,WAS vs. UTA,WAS,UTA,H,W,9,13,4,...,"Kanter, Enes",202683,1.2,0,0,john wall,202322,0.448513,874,15
128067,21400350,14-Dec-14,WAS vs. UTA,WAS,UTA,H,W,9,14,4,...,"Kanter, Enes",202683,1.4,1,2,john wall,202322,0.448513,874,15


In [39]:
# Total number of players
Player_Stats.shape

(281, 2)

In [40]:
# Total number of shots
Shotlog_1415.shape

(128069, 27)

In [41]:
# 2.	Convert the “date” variable to a date type variable and calculate summary statistics for the “shot_clock” variable

In [42]:
# Store "date" as a date type variable and produce summary statistics for the variable
Shotlog_1415['date']=pd.to_datetime(Shotlog_1415['date'])
Shotlog_1415['date'].describe()

<ipython-input-42-79e8e4f11943>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Shotlog_1415['date']=pd.to_datetime(Shotlog_1415['date'])


,date
count,128069
mean,2014-12-28 08:50:03.031178496
min,2014-10-28 00:00:00
25%,2014-11-28 00:00:00
50%,2014-12-27 00:00:00
75%,2015-01-26 00:00:00
max,2015-03-04 00:00:00


In [43]:
# Summary statistics for the "shot_clock" variable
Shotlog_1415['shot_clock'].describe()

,shot_clock
count,122502.000000
mean,12.453344
std,5.763265
min,0.000000
25%,8.200000
50%,12.300000
75%,16.675000
max,24.000000


In [44]:
# 3.	Create a lagged variable “lag_shot_hit” to indicate the result of the previous shot by the same player at the same game.
# Hint: In this dataset, the variable “match” may not be able to uniquely identify each game; you can use “game_id” instead. You can sort the data by shot number for each player to create the lagged variable

In [45]:
# Create a lagged variable to indicate the result of the previous shot by the same player in the same game:
#     - first sort the current shot outcome and match;
#     - group the data by player and match and use the "shift" command to create a lag variable

Shotlog_1415['lag_shot_hit']=Shotlog_1415.sort_values(by=['shot_number'], ascending=[True]).groupby(['player_id','game_id'])['current_shot_hit'].shift(1)
Shotlog_1415.head()

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,closest_defender_id,closest_def_dist,current_shot_hit,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game,lag_shot_hit
0,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,1,1,...,201572,6.6,1,2,al horford,201143,0.541259,715,10,NaN
1,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,2,1,...,201572,5.6,0,0,al horford,201143,0.541259,715,10,1.0
2,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,3,1,...,201572,4.7,0,0,al horford,201143,0.541259,715,10,0.0
3,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,4,1,...,201572,5.8,0,0,al horford,201143,0.541259,715,10,0.0
4,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,5,2,...,201572,6.4,0,0,al horford,201143,0.541259,715,10,0.0


In [46]:
# 4.	Create a variable “error” to indicate the prediction error for each shot and a variable “lagerror” for the prediction error for the previous shot.
# The “error” variable is defined as the difference between the outcome of the current shot and the average success rate (“average_hit”) and
# the “lagerror” variable is defined as the difference between the outcome of the previous shot and the average success rate

In [47]:
# Creation of a variable “error” to indicate the prediction error for each shot and
# a variable “lagerror” for the prediction error for the previous shot

Shotlog_1415['error']=Shotlog_1415['current_shot_hit']-Shotlog_1415['average_hit']
Shotlog_1415['lagerror']=Shotlog_1415['lag_shot_hit']-Shotlog_1415['average_hit']
Shotlog_1415.head()

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,current_shot_hit,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game,lag_shot_hit,error,lagerror
0,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,1,1,...,1,2,al horford,201143,0.541259,715,10,NaN,0.458741,NaN
1,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,2,1,...,0,0,al horford,201143,0.541259,715,10,1.0,-0.541259,0.458741
2,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,3,1,...,0,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259
3,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,4,1,...,0,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259
4,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,5,2,...,0,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259


In [48]:
# 5.	Calculate summary statistics for the “error” and “lagerror” variables

In [49]:
Shotlog_1415['error'].describe()

,error
count,1.280690e+05
mean,1.862089e-17
std,4.949640e-01
min,-7.124682e-01
25%,-4.491979e-01
50%,-3.850837e-01
75%,5.395973e-01
max,6.914894e-01


In [50]:
Shotlog_1415['lagerror'].describe()

,lagerror
count,113726.000000
mean,0.006303
std,0.496035
min,-0.712468
25%,-0.449198
50%,-0.382143
75%,0.542254
max,0.691489


## Part 2 - Conditional Probability and Autocorrelation

In [51]:
# 1.	Create a dummy variable “conse_shot” that indicates a player made consecutive shots

In [52]:
Shotlog_1415['conse_shot'] = np.where((Shotlog_1415['current_shot_hit']==1)&(Shotlog_1415['lag_shot_hit']==1), 1, 0)
Shotlog_1415.head()

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game,lag_shot_hit,error,lagerror,conse_shot
0,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,1,1,...,2,al horford,201143,0.541259,715,10,NaN,0.458741,NaN,0
1,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,2,1,...,0,al horford,201143,0.541259,715,10,1.0,-0.541259,0.458741,0
2,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,3,1,...,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259,0
3,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,4,1,...,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259,0
4,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,5,2,...,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259,0


In [53]:
# 2.	Create a dataframe “Player_Prob” for the probability of making the previous shot and the joint probability for making both the previous and current shots.
# Name the probability of making the previous shot “average_lag_hit” and the probability of making both shots “conse_shot_hit”

In [54]:
# Dataframe for the probability of making the previous shot and
# the joint probability for making both the previous and current shots
Player_Prob=Shotlog_1415.groupby(['shoot_player'])[['conse_shot','lag_shot_hit']].mean().reset_index()
Player_Prob.rename(columns={'lag_shot_hit':'average_lag_hit'}, inplace=True)
Player_Prob.rename(columns={'conse_shot':'conse_shot_hit'}, inplace=True)
Player_Prob.head()

,shoot_player,conse_shot_hit,average_lag_hit
0,aaron brooks,0.153298,0.418000
1,aaron gordon,0.201923,0.532468
2,al farouq aminu,0.162791,0.465686
3,al horford,0.262937,0.537994
4,al jefferson,0.207500,0.480000


In [55]:
# 3.	In the “Player_Prob” dataframe, calculate the conditional probability “conditional_prob” for a player to make a shot given that he made the previous shot

In [56]:
# Conditional probability for a player to make a shot given that he made the previous shot
Player_Prob['conditional_prob']=Player_Prob['conse_shot_hit']/Player_Prob['average_lag_hit']
Player_Prob.head()

,shoot_player,conse_shot_hit,average_lag_hit,conditional_prob
0,aaron brooks,0.153298,0.418000,0.366741
1,aaron gordon,0.201923,0.532468,0.379221
2,al farouq aminu,0.162791,0.465686,0.349572
3,al horford,0.262937,0.537994,0.488736
4,al jefferson,0.207500,0.480000,0.432292


In [57]:
# 4.	Merge the “Player_Prob” dataframe into the “Player_Stats” dataframe

In [58]:
Player_Stats=pd.merge(Player_Prob, Player_Stats, on=['shoot_player'])
Player_Stats.head()

,shoot_player,conse_shot_hit,average_lag_hit,conditional_prob,average_hit
0,aaron brooks,0.153298,0.418000,0.366741,0.415330
1,aaron gordon,0.201923,0.532468,0.379221,0.528846
2,al farouq aminu,0.162791,0.465686,0.349572,0.430233
3,al horford,0.262937,0.537994,0.488736,0.541259
4,al jefferson,0.207500,0.480000,0.432292,0.477500


In [59]:
# 5.	Calculate summary statistics for the probability for a player to make a shot (“average_hit”) and
# the conditional probability for a player to make a shot given that he made the previous one (“conditional_prob”) and
# the probability of players making consecutive shots (“conse_shot_hit”)

In [60]:
Player_Stats['average_hit'].describe()

,average_hit
count,281.000000
mean,0.451545
std,0.059392
min,0.308511
25%,0.413223
50%,0.446078
75%,0.480480
max,0.712468


In [61]:
Player_Stats['conditional_prob'].describe()

,conditional_prob
count,281.000000
mean,0.380233
std,0.062320
min,0.225801
25%,0.336689
50%,0.381570
75%,0.422801
max,0.613209


In [62]:
Player_Stats['conse_shot_hit'].describe()

,conse_shot_hit
count,281.000000
mean,0.176987
std,0.047943
min,0.076190
25%,0.144543
50%,0.171625
75%,0.203512
max,0.422392


In [63]:
# 6.	Perform a t-test for the statistical significance on the difference between conditional probability and unconditional probability of making a shot

In [64]:
sp.stats.ttest_ind(Player_Stats['conditional_prob'], Player_Stats['average_hit'])

<ipython-input-64-5126942b8bdd>:1: DeprecationWarning: Please import `ttest_ind` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  sp.stats.ttest_ind(Player_Stats['conditional_prob'], Player_Stats['average_hit'])


TtestResult(statistic=-13.885932802814914, pvalue=6.925846314604592e-38, df=560.0)

In [65]:
# 7.	Calculate the first order autocorrelation coefficient on making a shot (correlation coefficient between making the current shot and the previous shot) for the entire shotlog dataset

In [66]:
Shotlog_1415['current_shot_hit'].corr(Shotlog_1415['lag_shot_hit'])

-0.010502388301693177

In [67]:
# 8.	Calculate the first order autocorrelation coefficient on making a shot for each player.
# Display the top ten players with the highest first order autocorrelation coefficient

In [68]:
Autocorr_Hit=Shotlog_1415.groupby('shoot_player')[['current_shot_hit','lag_shot_hit']].corr().unstack().iloc[:,1].reset_index()
Autocorr_Hit.columns=Autocorr_Hit.columns.get_level_values(0)
Autocorr_Hit.rename(columns={'current_shot_hit':'autocorr'}, inplace=True)
Autocorr_Hit.sort_values(by=['autocorr'], ascending=[False]).head(10)

,shoot_player,autocorr
131,joey dorsey,0.334252
54,cole aldrich,0.174666
200,nate robinson,0.122107
267,tyler hansbrough,0.120608
7,alex len,0.118461
50,cj mccollum,0.115949
114,jason smith,0.105903
190,matt bonner,0.098577
143,jusuf nurkic,0.097465
195,mike miller,0.089366


## Part 3 - Regression Analyses

In [69]:
# Reg1: Run a linear least squares regression using the entire shotlog dataframe. Include the following control variables:
# •	Shot distance
# •	Number of dribbles
# •	Touch time
# •	Type of shot (“points” variable)
# •	Quarter of the game (as a categorical variable)
# •	Home or away game
# •	Shoot_player
# •	Closest defender
# •	Closest defender distance

In [70]:
# Reg1: linear least squares regression using the entire shotlog dataframe
# - Dependent variable: error
# - Independent variables:lagerror, shot_dist, dribbles, touch_time, points, quarter, home_away, shoot_player, closest_defender, and closest_def_dist

In [71]:
reg1 = sm.ols(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+C(points)+C(quarter)+home_away+shoot_player+closest_defender+closest_def_dist', data=Shotlog_1415).fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     9.002
Date:                Wed, 18 Dec 2024   Prob (F-statistic):               0.00
Time:                        22:12:37   Log-Likelihood:                -78065.
No. Observations:              113726   AIC:                         1.577e+05
Df Residuals:                  112960   BIC:                         1.650e+05
Df Model:                         765                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------

In [72]:
# Reg2: Run a weighted least squares regression using the entire shotlog dataframe. Include the same set of control variables as in Reg1.
#       The regression should be weighted by the number of shot per game (weight=1/shot_per_game)

In [73]:
# Reg2: Weighted least squares regression using the entire shotlog dataframe, weighted by shot_per_game.
# - Dependent variable: error
# - Independent variables:lagerror, shot_dist, dribbles, touch_time, points, quarter, home_away, shoot_player, closest_defender, and closest_def_dist

In [74]:
reg2 = sm.wls(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+C(points)+C(quarter)+home_away+shoot_player+closest_defender+closest_def_dist', weights=1/Shotlog_1415['shot_per_game'], data=Shotlog_1415).fit()
print(reg2.summary())

                            WLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.062
Model:                            WLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     9.738
Date:                Wed, 18 Dec 2024   Prob (F-statistic):               0.00
Time:                        22:13:25   Log-Likelihood:                -86948.
No. Observations:              113726   AIC:                         1.754e+05
Df Residuals:                  112960   BIC:                         1.828e+05
Df Model:                         765                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------

In [75]:
# Reg3_player: Run linear least squares regressions on individual players. Include the following control variables:
# •	Shot distance
# •	Number of dribbles
# •	Touch time
# •	Type of shot (“points” variable)
# •	Quarter of the game (as a categorical variable)
# •	Home or away game
# •	Closest defender distance

In [76]:
# Reg3_player: linear least squares regressions on individual players
# - Dependent variable: error
# - Independent variables:lagerror, shot_dist, dribbles, touch_time, points, quarter, home_away, and closest_def_dist

In [77]:
def Reg3_player(player):
    Shotlog_player=Shotlog_1415[Shotlog_1415.shoot_player==player]
    Reg3_player=sm.ols(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+C(points)+C(quarter)+home_away+closest_def_dist', data=Shotlog_player).fit()
    print(Reg3_player.summary())
    return;

In [78]:
Reg3_player('andrew wiggins')

                            OLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     5.602
Date:                Wed, 18 Dec 2024   Prob (F-statistic):           1.14e-08
Time:                        22:13:26   Log-Likelihood:                -500.04
No. Observations:                 738   AIC:                             1024.
Df Residuals:                     726   BIC:                             1079.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.2003      0.057  

In [79]:
Reg3_player('stephen curry')

                            OLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     6.498
Date:                Wed, 18 Dec 2024   Prob (F-statistic):           1.79e-10
Time:                        22:13:26   Log-Likelihood:                -625.19
No. Observations:                 910   AIC:                             1274.
Df Residuals:                     898   BIC:                             1332.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.2196      0.057  

In [80]:
Reg3_player('james harden')

                            OLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     4.952
Date:                Wed, 18 Dec 2024   Prob (F-statistic):           1.61e-07
Time:                        22:13:26   Log-Likelihood:                -691.15
No. Observations:                 995   AIC:                             1406.
Df Residuals:                     983   BIC:                             1465.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.1003      0.053  

In [81]:
Reg3_player('russell westbrook')

                            OLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     5.529
Date:                Wed, 18 Dec 2024   Prob (F-statistic):           1.32e-08
Time:                        22:13:26   Log-Likelihood:                -631.81
No. Observations:                 923   AIC:                             1288.
Df Residuals:                     911   BIC:                             1346.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0958      0.051  

In [82]:
# Reg4_wls_player: Run weighted least squares regressions on individual players. Include the same set of control variables as in Reg3.
#     The regression should be weighted by the number of shot per game (weight=1/shot_per_game)

In [83]:
# Reg4_wls_player: weighted least squares regressions on individual players, weighted by shot_per_game.
# - Dependent variable: error
# - Independent variables:lagerror, shot_dist, dribbles, touch_time, points, quarter, home_away, and closest_def_dist

In [84]:
def Reg4_wls_player(player):
    Shotlog_player=Shotlog_1415[Shotlog_1415.shoot_player==player]
    Reg4_wls_player=sm.wls(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+C(points)+C(quarter)+home_away+closest_def_dist', weights=1/Shotlog_player['shot_per_game'], data=Shotlog_player).fit()
    print(Reg4_wls_player.summary())
    return;

In [85]:
Reg4_wls_player('reggie jackson')

                            WLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.083
Model:                            WLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     4.477
Date:                Wed, 18 Dec 2024   Prob (F-statistic):           6.58e-07
Time:                        22:13:26   Log-Likelihood:                -440.81
No. Observations:                 604   AIC:                             907.6
Df Residuals:                     591   BIC:                             964.9
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.1963      0.081  

In [86]:
Reg4_wls_player('alonzo gee')

                            WLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.239
Model:                            WLS   Adj. R-squared:                  0.146
Method:                 Least Squares   F-statistic:                     2.564
Date:                Wed, 18 Dec 2024   Prob (F-statistic):            0.00714
Time:                        22:13:27   Log-Likelihood:                -68.055
No. Observations:                 102   AIC:                             160.1
Df Residuals:                      90   BIC:                             191.6
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -0.1300      0.564  

In [87]:
Reg4_wls_player('cole aldrich')

                            WLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.162
Model:                            WLS   Adj. R-squared:                  0.109
Method:                 Least Squares   F-statistic:                     3.074
Date:                Wed, 18 Dec 2024   Prob (F-statistic):            0.00212
Time:                        22:13:27   Log-Likelihood:                -110.28
No. Observations:                 153   AIC:                             240.6
Df Residuals:                     143   BIC:                             270.9
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0614      0.170  

In [88]:
Reg4_wls_player('stephen curry')

                            WLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.078
Model:                            WLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     6.868
Date:                Wed, 18 Dec 2024   Prob (F-statistic):           3.42e-11
Time:                        22:13:27   Log-Likelihood:                -660.11
No. Observations:                 910   AIC:                             1344.
Df Residuals:                     898   BIC:                             1402.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.2125      0.057  